In [31]:
import pandas as pd
from scipy import stats

In [21]:
ab_users_data = pd.read_csv("/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-zakorko/ab_users_data.csv")
ab_orders = pd.read_csv("/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-zakorko/ab_orders.csv")
ab_products = pd.read_csv("/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-zakorko/ab_products.csv")

In [26]:
ab_users_data.head()

,user_id,order_id,action,time,date,group
0,964,1255,create_order,2022-08-26 00:00:19.000000,2022-08-26,0
1,965,1256,create_order,2022-08-26 00:02:21.000000,2022-08-26,1
2,964,1257,create_order,2022-08-26 00:02:27.000000,2022-08-26,0
3,966,1258,create_order,2022-08-26 00:02:56.000000,2022-08-26,0
4,967,1259,create_order,2022-08-26 00:03:37.000000,2022-08-26,1


In [27]:
ab_orders.head()

,order_id,creation_time,product_ids
0,1255,2022-08-26 00:00:19.000000,"{75, 22, 53, 84}"
1,1256,2022-08-26 00:02:21.000000,"{56, 76, 39}"
2,1257,2022-08-26 00:02:27.000000,"{76, 34, 41, 38}"
3,1258,2022-08-26 00:02:56.000000,"{74, 6}"
4,1259,2022-08-26 00:03:37.000000,"{20, 45, 67, 26}"


In [28]:
ab_products.head()

,product_id,name,price
0,1,сахар,150.0
1,2,чай зеленый в пакетиках,50.0
2,3,вода негазированная,80.4
3,4,леденцы,45.5
4,5,кофе 3 в 1,15.0


Для оценки эффективности новой системы рекомендаций мы можем рассмотреть следующие метрики:

Средний чек заказа(Целевая метрика): Сравнение среднего чека заказа между тестовой и контрольной группами. Больший средний чек в тестовой группе может указывать на то, что система рекомендаций способствует увеличению расходов пользователей.

Количество отменённых заказов(Прокси-метрика): Процент отменённых заказов в обеих группах. Меньший процент отмен в тестовой группе может указывать на улучшение качества выбора продуктов благодаря рекомендациям.

Среднее количество товаров в заказе(Прокси-метрика): Сравнение среднего количества товаров в заказе между группами. Большее количество товаров в заказах тестовой группы может свидетельствовать о том, что рекомендации помогают пользователям находить больше нужных товаров.

In [53]:
# Подготовка данных для анализа метрик

# 1. Объединение данных о заказах с информацией о пользователях
merged_orders_users = pd.merge(ab_orders, ab_users_data, on="order_id")

# 2. Конвертация строк с идентификаторами продуктов в списки
merged_orders_users['product_ids'] = merged_orders_users['product_ids'].apply(lambda x: [int(i) for i in x.strip('{}').split(',')])

# 3. Подсчет общей стоимости заказа
# Сначала создаем словарь для быстрого доступа к ценам продуктов
product_prices = dict(zip(ab_products['product_id'], ab_products['price']))

# Функция для подсчета общей стоимости заказа
def calculate_order_total(product_ids):
    return sum(product_prices[pid] for pid in product_ids)

# Применяем функцию к каждому заказу
merged_orders_users['order_total'] = merged_orders_users['product_ids'].apply(calculate_order_total)

# 4. Рассчет метрик
# Средний чек заказа для каждой группы
average_order_value = merged_orders_users[merged_orders_users['action'] == 'create_order'].groupby('group')['order_total'].mean()

# Процент отменённых заказов в каждой группе
cancellation_rate = merged_orders_users[merged_orders_users['action'] == 'cancel_order'].groupby('group')['order_id'].nunique() / merged_orders_users.groupby('group')['order_id'].nunique()

# Среднее количество товаров в заказе
average_items_per_order = merged_orders_users[merged_orders_users['action'] == 'create_order'].groupby('group')['product_ids'].apply(lambda x: x.str.len().mean())

# Собираем метрики в один DataFrame для удобства сравнения
metrics = pd.DataFrame({
    "Средник чек заказа": average_order_value,
    "Процент отменённых заказов": cancellation_rate,
    "Среднее количество товаров в заказе": average_items_per_order
})

metrics


,Средник чек заказа,Процент отменённых заказов,Среднее количество товаров в заказе
group,,,
0,381.285768,0.050963,3.349907
1,369.622912,0.052506,3.355609


Средний чек заказа немного выше в контрольной группе, что может указывать на то, что новая система рекомендаций не способствовала увеличению расходов пользователей.

Процент отменённых заказов немного выше в тестовой группе, что может указывать на то, что новая система рекомендаций не помогла снизить количество отмененных заказов.

Среднее количество товаров в заказе практически одинаково в обеих группах, что указывает на отсутствие значительного влияния новой системы рекомендаций на количество покупаемых товаров.

Для более глубокой оценки результатов АБ-теста, необходимо провести статистический анализ данных. 
Особенно важно проверить статистическую значимость различий между группами по целевой метрике (Средний чек заказа), используя t-тест.

In [56]:
# Извлекаем значения среднего чека заказа для каждой группы
group_0_order_values = merged_orders_users[(merged_orders_users['group'] == 0) & (merged_orders_users['action'] == 'create_order')]['order_total']
group_1_order_values = merged_orders_users[(merged_orders_users['group'] == 1) & (merged_orders_users['action'] == 'create_order')]['order_total']

# Проведение t-теста для сравнения средних двух независимых выборок
t_test_result = stats.ttest_ind(group_0_order_values, group_1_order_values, equal_var=False)  # Welch's t-test

t_test_result


Ttest_indResult(statistic=1.4835135849437482, pvalue=0.1380295748494135)

Выводы:

t-значение показывает разницу между средними значениями двух групп. В данном случае t-значение равно 1.48, что указывает на некоторую разницу между группами.

p-значение говорит нам о вероятности получить такие или еще более выраженные различия между группами, если бы на самом деле никаких различий не было (нулевая гипотеза). P-значение 0.138 превышает обычный порог статистической значимости (0.05), что означает, что мы не можем отвергнуть нулевую гипотезу о том, что средний чек заказа не различается между группами.

На основе проведённого статистического анализа мы не можем утверждать, что внедрение новой системы рекомендаций оказало значимое влияние на средний чек заказа в приложении по доставке продуктов. Различия между контрольной и тестовой группами не являются статистически значимыми.